In [ ]:
import geopy
from geopy.distance import distance
import pandas as pd

In [ ]:
pharmacies = pd.read_csv('../data/geocoded_pharmacies.csv')
records = pd.read_csv('../data/version3.csv')

In [5]:
distances = []
for i, row in pharmacies.iterrows():
    for j, row2 in records.iterrows():
        if pd.isna(row.coded_lat) or pd.isna(row.coded_long):
            continue
        if pd.isna(row2.coded_lat) or pd.isna(row2.coded_long):
            continue
        d = distance((row.coded_lat, row.coded_long), (row2.coded_lat, row2.coded_long))
        distances.append(d.km)

sum(d) / len(d)

AttributeError: 'Series' object has no attribute 'lat'